In [2]:
import pandas as pd
from constraint import Problem, AllDifferentConstraint

In [3]:
muscle_map_df = pd.read_csv('muscle_map.csv')
# I want to get all the distinct values from column Body Part
muscle_map_df['Muscle'] = muscle_map_df['Muscle'].apply(lambda x: x.lower())
muscle_map_df.head()

,Muscle,Body Part
0,hamstrings (below knees),Calves
1,"erector spinae, cervicis & capitis fibers",Back
2,triceps brachii,Upper Arms
3,gracilis,Thighs
4,"pectoralis major, sternal",Chest


In [4]:
muscle_map_df['Body Part'].unique()

array(['Calves', 'Back', 'Upper Arms', 'Thighs', 'Chest', 'Hips',
       'Forearms', 'Shoulders', 'Waist', 'Neck', 'Thigs', 'neck'],
      dtype=object)

In [5]:
exercises_df = pd.read_csv('exrx.csv')
# remove the force column 
exercises_df = exercises_df.drop(columns=['force','stabilizer_muscles','antagonist_stabilizer_muscles','synergist_muscles','dynamic_stabilizer_muscles'])
# Transform all the target muscles into a single string in lower case. It is a string array by default
exercises_df['target_muscles'] = exercises_df['target_muscles'].apply(lambda x: x[2:-2].lower())
# # Convert all muscle names to lowercase in the mapping dictionary
muscle_map_dict = dict(zip(muscle_map_df['Muscle'], muscle_map_df['Body Part']))

# Use map and lambda to create the 'major muscle' column in 'exercises_df'
exercises_df['major_muscle'] = exercises_df['target_muscles'].apply(lambda x: muscle_map_dict.get(x, 'Unknown'))
#remove the rows with 'Unknown' major muscle
exercises_df = exercises_df[exercises_df['major_muscle'] != 'Unknown']
#remove the rows with 'Specialized' utility
exercises_df = exercises_df[exercises_df['utility'] != 'Specialized']
#remove the rows with Null utility
exercises_df = exercises_df[exercises_df['utility'].notnull()]

exercises_df['utility'] = exercises_df['utility'].apply(lambda x: 'Basic or Auxiliary' if x == 'Auxiliary or Basic' else x)
exercises_df.head()

,exercise,utility,mechanics,target_muscles,major_muscle
0,Barbell Triceps Extension,Basic or Auxiliary,Isolated,triceps brachii,Upper Arms
1,Cable Bar Lying Triceps Extension,Basic,Isolated,triceps brachii,Upper Arms
2,Smith Rear Delt Row,Basic or Auxiliary,Compound,"deltoid, posterior",Shoulders
3,Lever Hack Squat (plate loaded),Basic or Auxiliary,Compound,gluteus maximus,Hips
4,Sled Lying Reverse Calf Press (plate loaded),Basic or Auxiliary,Isolated,tibialis anterior,Calves


In [6]:
# find rows that have compound mechanics and auxiliary utility
exercises_df[(exercises_df['mechanics'] == 'Compound') & (exercises_df['utility'] == 'Auxiliary')]

,exercise,utility,mechanics,target_muscles,major_muscle
17,Barbell Lateral Step-up,Auxiliary,Compound,quadriceps,Thighs
27,Suspended Power Pull,Auxiliary,Compound,"back, general",Back
28,Cable Split Squat (belt),Auxiliary,Compound,gluteus maximus,Hips
53,Barbell Walking Lunge,Auxiliary,Compound,gluteus maximus,Hips
54,Barbell Side Split Squat,Auxiliary,Compound,gluteus maximus,Hips
...,...,...,...,...,...
1433,Cable Standing Cross Row,Auxiliary,Compound,"deltoid, posterior",Shoulders
1438,Smith Hack Squat,Auxiliary,Compound,quadriceps,Thighs
1441,Cable Rear Lunge,Auxiliary,Compound,gluteus maximus,Hips
1442,Sled Single Leg 45° Leg Press,Auxiliary,Compound,quadriceps,Thighs


In [86]:
# output the dataframe to a csv file
exercises_df.to_csv('exercises_cleaned.csv', index=False)